### Set up

In [ ]:
# file system navigation
from pathlib import Path

# data transformation
import pandas as pd

# nlp
import spacy
from spacy_cld import LanguageDetector

### Data preparation

In [ ]:
articles = pd.read_csv(Path.cwd() / "data" / "articles.csv")

In [ ]:
articles.shape

#### Find and remove duplicates based on author and title

In [ ]:
articles = articles\
    .drop_duplicates(subset=["author", "title"], keep="first")

In [ ]:
articles.shape

#### Convert claps to integer

In [ ]:
articles["claps"] = articles["claps"].apply(lambda s: int(float(s[:-1]) * 1000) if s[-1] == "K" else int(s))

In [ ]:
articles.head()

#### Remove articles, which are not written in english

In [ ]:
nlp = spacy.load("en")
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)

In [ ]:
def get_en_score(df, col_name):
    
    data = df.copy()
    data["en_score"] = data[col_name].apply(lambda x: nlp(x)._.language_scores.get("en", 0.0))
    
    return data

In [ ]:
unique_articles_en = articles \
        .pipe(get_en_score, "text") \
        .query("en_score > 0.9") \
        .drop("en_score", axis=1)

In [ ]:
unique_articles_en.shape

In [ ]:
unique_articles_en.sort_values("claps", ascending=False).head(100)

#### Prepare and save data for labeling (author, claps and link only) as csv and for further processing as parquet

In [ ]:
unique_articles_en\
    .sort_values(by="claps", ascending=False)\
    .loc[:, ["author", "claps", "link"]]\
    .to_csv(Path.cwd() / "data" / "processed" / "articles_link_only.csv", sep=";")

In [ ]:
unique_articles_en \
    .to_parquet(Path.cwd() / "data" / "processed" / "unique_articles.parquet", engine="pyarrow")

#### Add labels

In [ ]:
labels = pd.read_csv(Path.cwd() / "data" / "labels.csv", sep=";")

In [ ]:
labeled_data = pd.merge(unique_articles_en, labels, how="inner", left_index=True, right_on="id").drop("id", axis=1)

In [ ]:
labeled_data.sort_values("claps", ascending=False)[["author", "claps", "interesting"]].head()

In [ ]:
labeled_data \
    .to_parquet(Path.cwd() / "data" / "processed" / "train_data.parquet", engine="pyarrow")

In [ ]:
labeled_data_text_only = labeled_data[["text", "interesting"]]

labeled_data_text_only.to_csv(Path.cwd() / "data" / "processed" / "train_data_text_only.csv", index=False)

### Experimentation

#### Remove special characters

In [ ]:
unique_articles_en = pd.read_csv(os.path.join(wd, "data", "processed", "unique_articles.csv"))

In [ ]:
def get_raw_text(df, col_name):
    
    return df\
            .loc[:, col_name]\
            .str\
            .cat(sep = "|")\
            .lower()

In [ ]:
unique_chars_title = set(get_raw_text(unique_articles_en, "title"))
print(f"There are a total of {len(unique_chars_title)} unique characters in column title:")
print(unique_chars_title)

In [ ]:
unique_chars_text = set(get_raw_text(unique_articles_en, "text"))
print(f"There are a total of {len(unique_chars_text)} unique characters in column text:")
print(unique_chars_text)

In [ ]:
text = unique_articles_en.iloc[0, 4]

In [ ]:
doc = nlp(text)

In [ ]:
doc

In [ ]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [ ]:
for token in doc:
    print(token.text)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)